In [1]:
%pylab inline
from scipy.linalg import hadamard as hadamard
from sympy import isprime as isprime
from math import gcd as GCD
from scipy.sparse.coo import coo_matrix as sparsemat
from scipy.linalg import eigvalsh as eigenval
from scipy.sparse.linalg import eigsh as sparse_eigenval
from scipy.sparse.linalg import svds as sparsesvd

Populating the interactive namespace from numpy and matplotlib


In [2]:
    "parameters of the system and operators"

    
    "Number to be decomposed"
    N=21
    
    "control and target registers size"
    L=math.ceil(math.log2(N))
    control_size=2**(2*L)
    target_size=2**L
    
    
    """
    superposition of dimension 2**2L x 2**2L, to be constructed only once.
    Avoided memorization of the H_2L Hadamard operator, since it's not sparse.
    control_superposition= H^2L |0>
    """  
    control_superposition=np.ones(control_size)/control_size
    
    print("qubits required= ",3*L)

qubits required=  15


In [3]:
def find_coprime(N):     
    """
            Find a coprime of N for N>2
            Parameters
            ----------
            N:  Integer
                Number to find the coprime of

    """
    if(N<3):
        raise ValueError("Illegal argument: coprimes exist for N>2")
    Y=randint(2,N)
    used=[]
    while 1:
        a=GCD(Y,N)
        if (a>1):
            #this Y is not coprime
            used.append(Y)
            while(Y in used):
                Y=randint(2,N)
        else: return Y

        
def decimal_to_state(m,nqubit):
    '''
        Return binary representation of m as array of nqubit qubits

            Parameters
            ----------                
            m:   Integer
                number to be representend in binary form
                
            nqubit: Integer
                Total number of qubits used for representation
                    
    '''
    
    arr=binary_repr(m)
    arr=[int(i) for i in arr]
    if(len(arr)>nqubit):
        raise ValueError(str(nqubit)+" are not enough qubits to store the number "+str(m))
    if(len(arr)==nqubit): return arr
    return list(np.zeros(nqubit-len(arr),dtype=int16))+ arr


def to_decimal(array):
    '''
        Return decimal representation of the array storing a number in binary form

        Example: input [1,0,1,0,1] returns 21 
            Parameters
            ----------                
            array: List
                array containing binary representation of a number
                
                    
    '''

    size=len(array)
    return int(np.sum([array[i] *2**(size-i-1)  for i in range(size)]))

In [4]:
def notchosen(chosen,system_size):
    """
            Return array containing the qubit NOT in the partition
            
            Parameters
            ----------                
            chosen:   List
                List of the qubits selected as partition, in the form [1,3,7,..]
                
            system_size: Integer
                Total number of qubits
                    
    """
    
    notchosen=list(set(list(range(system_size)))-set(chosen))
    notchosen.sort()
    return notchosen



def bipartition_state(in_idx,out_idx,chosen,notchosen):
    """
            Return array representing index of the state expressed through in_idx and out_idx
            
            Parameters
            ----------
            in_index:  Integer
                Index of the state restricted on the partition in decimal notation
                
            out_idx:   Integer
                Index of the state restricted on the remaining part in decimal notation    
                       
            chosen:   List
                List of the qubits selected as partition, in the form [1,3,7,..]
                
            notchosen:   List
                List of the qubits not selected as partition, in the form [2,4,5,6,8,..]
                    
    """
        
    part_size=size(chosen)
    out_size=size(notchosen)
    system_size=part_size+out_size
    
    part_state=list(map(lambda x, y:(x,y), decimal_to_state(in_idx,part_size), chosen))
    out_state=list(map(lambda x, y:(x,y), decimal_to_state(out_idx,out_size), notchosen))
   
    total=part_state+out_state
    total.sort(key=lambda x: x[1])
    return [elem[0] for elem in total]
    
def psi(k,Y,N):
    L=int(ceil(log2(N)))
    if(k>2*L):
        raise ValueError(str(k)+"th computational step does not make sense in a "+str(2*L)+" qubits control register")
    
    data=np.ones(2**k,dtype=int8)
    row=[m*2**L+(Y**m%N) for m in range(2**k)]
    col=np.zeros(2**k,dtype=int8)
    psi=sparsemat((data,(row,col)),shape=(2**(k+L),1))
    return (psi/2**k).tocsc()



def create_W_slow(psi,chosen):    
    size=int(ceil(log2(shape(psi)[0])))
    chosen_size=len(chosen)
    if(size<max(chosen)):
        raise ValueError(str(max(chosen))+ 'qubit not present in a '+str(size)+" qubits system")

    not_chosen=notchosen(chosen,size)
    notchosen_size=size-chosen_size
    W=[ [   psi[to_decimal(bipartition_state(i,j,chosen,not_chosen)),0] for j in range(2**notchosen_size)]for i in range(2**chosen_size) ]
    return sparsemat(W,shape=(2**chosen_size,2**notchosen_size))

In [5]:
%%time
kloc,Yloc,Nloc=(2,20,21)
ps=psi(kloc,Yloc,Nloc)
#print([j for j,i in enumerate(ps.toarray()) if i!=0])
W=create_W_slow(ps,[0,2])
#k=6, 13 s, k=7 24 s, k=8 49, k=10 circa 3 min e 15 sec

CPU times: user 25.3 ms, sys: 0 ns, total: 25.3 ms
Wall time: 28.7 ms


In [96]:
def create_W(k,Y,N,chosen):
    '''
    creates W directly
    
    '''
    
    L=int(ceil(log2(N)))
    if(k>2*L):
        raise ValueError(str(k)+"th computational step does not make sense in a "+str(2*L)+" qubits control register")
    
    #nonzero elements of psi in binary form
    nonzeros=[decimal_to_state(m*2**L+(Y**m%N),k+L) for m in range(2**k)]
    not_chosen=notchosen(chosen,k+L)  

    indexes=[ (to_decimal(split_components(i,chosen)),to_decimal((split_components(i,not_chosen)))) for i in nonzeros]
    row=[elem[0] for elem in indexes]
    col=[elem[1] for elem in indexes]
    data=np.ones(2**k)/sqrt(2**k)
    
    return sparsemat((data,(row,col)), shape=(2**len(chosen),2**len(not_chosen))    ).tocsc()

def split_components(array,chosen):
    '''
    Given an input array and selected components returns two arrays.
    The first array contains only the chosen components and the other the remainders
    '''
    
    if( max(chosen) not in range(len(array)))  :
        raise ValueError('the chosen '+str(max(chosen))+' bit is not present in a '+str(len(array))+' bits register')
    return [array[i] for i in chosen]

def entanglement_entropy(W):
    if(shape(W)[0]<shape(W)[1]):
        A=dot(W,W.T)
    else:
        A=dot(W.T,W)
    
    eigs=eigenval(A.toarray())
    return -np.sum([i*log2(i) for i in eigs if i>0])

def entanglement_entropy_svd(W):
    eigs=(numpy.linalg.svd(W.toarray(),compute_uv=False,full_matrices=False))**2
    return -np.sum([i*log2(i) for i in eigs if i>0])

def entanglement_entropy_sparse(W):
    if(shape(W)[0]<shape(W)[1]):
        A=dot(W,W.T)
    else:
        A=dot(W.T,W)
    
    eigs=sparse_eigenval(A,k=shape(A)[0]-1,which='LM',return_eigenvectors=False)
    return -np.sum([i*np.log2(i) for i in eigs if i>0])

def entanglement_entropy_svd_sparse(W):
    eigs=sparsesvd(W,k=min(shape(W))-1,which='LM',return_singular_vectors=False, tol=1e-16)
    eigs=eigs*eigs
    return -np.sum([i*math.log2(i) for i in eigs if i>0])

Hp: A=W.T * W is Hermitian.
Eigenvals of A

In [100]:
%%time
#with eigvalsh (hermitian) N=39 needs 6.5sec, N=55 needs 12sec,N=99 needs 6 mins adn 15 sec
N=39  #39 candidate[4]
k=2*int(ceil(log2(N)))
new_S1=[entanglement_entropy(create_W(k,Y,N,[i for i in range(k)])) for Y in range(2,N) if(GCD(Y,N)==1)]

CPU times: user 7.26 s, sys: 0 ns, total: 7.26 s
Wall time: 7.26 s


SVD of W

In [101]:
%%time
#with svd (hermitian) N=39 needs 7.1sec, N=55 needs 12sec  ,N=99 needs 6 mins and 13 sec
N=39                 #39 candidate[4],   55 candidate[7],  99 candidate[19]
k=2*int(ceil(log2(N)))
new_S2=[entanglement_entropy_svd(create_W(k,Y,N,[i for i in range(k)])) for Y in range(2,N) if(GCD(Y,N)==1)]

CPU times: user 11.8 s, sys: 288 ms, total: 12.1 s
Wall time: 7.64 s


Hp: A Hermitian.  
almost all eigenvals of A keeping it as sparse matrix

In [102]:
%%time
#with svd (hermitian) N=39 needs 6.6sec, N=55 needs 12sec  ,N=99 needs 
N=55                 #39 candidate[4],   55 candidate[7],  99 candidate[19]
k=2*int(ceil(log2(N)))
new_S3=[entanglement_entropy_sparse(create_W(k,Y,N,[i for i in range(k)])) for Y in range(2,N) if(GCD(Y,N)==1)]

CPU times: user 13.8 s, sys: 20 ms, total: 13.8 s
Wall time: 13.6 s


almost all singval of W keeping it as sparse matrix

In [103]:
%%time
#with svd (hermitian) N=39 needs 6.7sec, N=55 needs 12sec  ,N=99 needs 
N=39                 #39 candidate[4],   55 candidate[7],  99 candidate[19]
k=2*int(ceil(math.log2(N)))
new_S4=[entanglement_entropy_svd_sparse(create_W(k,Y,N,[i for i in range(k)])) for Y in range(2,N) if(GCD(Y,N)==1)]

CPU times: user 11.9 s, sys: 291 ms, total: 12.1 s
Wall time: 7.79 s


In [104]:
import pickle

with open("heatmap.txt", "rb") as fp:
    newdata = pickle.load(fp)

old_S=[i[2] for i in newdata[4]]

In [107]:
relative_error1=[abs(i-j)/i for i,j in zip(old_S,new_S1) if i>0]
relative_error2=[abs(i-j)/i for i,j in zip(old_S,new_S2) if i>0]
relative_error3=[abs(i-j)/i for i,j in zip(old_S,new_S3) if i>0]
relative_error4=[abs(i-j)/i for i,j in zip(old_S,new_S4) if i>0]

print(max(relative_error1),max(relative_error2),max(relative_error3),max(relative_error4))
#print([ (i,j) for i,j in zip(old_S,relative_error) if i>0])

2.477511997076467e-16 2.477511997076467e-16 1.7268314379490548 6.217248937900877e-15


test for separable states

In [12]:
ps=np.zeros(2**7)
ps=(sparsemat(ps).T).tocsc()
res=0
for j in range(2**7):    
    ps[j]=1    
    for i in range(int(ceil(log2(shape(ps)[0])))):
        test_W=create_W_slow(ps,[i])
        res+=entanglement_entropy(test_W)
    ps[j]=0
    
print(res)

0.0


In [23]:
np.median([2,2,4,4])

3.0